In [3]:
import sqlite3
import pandas as pd
import requests

# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [11]:
API_KEY = 'fsq3hhg8hTQCfL/aPuz1PSLab+R48Pg6oojSQ2NqplkdW8A='  

def fetch_pois_from_foursquare(lat, lon, radius=1000, limit=50):
    """
    Fetch POIs from Foursquare around a given latitude and longitude using the Foursquare Places API.
    
    Parameters:
    - lat, lon (float): Latitude and longitude of the center point.
    - radius (int): Radius in meters to search within.
    - limit (int): Maximum number of results to return.
    
    Returns:
    - list of dicts: Each dict contains POI information.
    """
    url = "https://api.foursquare.com/v3/places/search"
    headers = {
        "Accept": "application/json",
        "Authorization": API_KEY
    }
    params = {
        'll': f"{lat},{lon}",
        'radius': radius,
        'limit': limit
    }
    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        venues = response.json()['results']
        return [{
            'station_lat': lat,
            'station_lon': lon,
            'poi_name': venue['name'],
            'poi_lat': venue['geocodes']['main']['latitude'],
            'poi_lon': venue['geocodes']['main']['longitude']
        } for venue in venues]
    else:
        return []

# Container for all POIs data
all_pois = []

# Iterate over each bike station in the DataFrame to fetch nearby POIs
for index, row in stations_df.iterrows():
    pois = fetch_pois_from_foursquare(row['latitude'], row['longitude'])
    all_pois.extend(pois)

# Convert the collected POI data into a DataFrame
pois_df = pd.DataFrame(all_pois)

# Quick check to see the first few rows of the collected POIs
print(pois_df.head())



NameError: name 'stations_df' is not defined

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [ ]:
API_KEY = 'fsq3hhg8hTQCfL/aPuz1PSLab+R48Pg6oojSQ2NqplkdW8A='
stations_df = pd.DataFrame({
    'station_id': [1, 2, 3],
    'latitude': [41.387015, 41.391947, 41.396992],
    'longitude': [2.170047, 2.164485, 2.165105]
})

def fetch_and_parse_pois(lat, lon, radius=1000, limit=50):
    """
    Fetches POIs from Foursquare around a given latitude and longitude.
    Parses the response to extract names, locations, and ratings of venues.
    """
    url = "https://api.foursquare.com/v3/places/search"
    headers = {"Authorization": API_KEY}
    params = {'ll': f"{lat},{lon}", 'radius': radius, 'limit': limit, 'sort': 'RATING'}
    response = requests.get(url, headers=headers, params=params)
    
    if response.status_code == 200:
        venues = response.json()['results']
        parsed_venues = []
        for venue in venues:
            venue_details = {
                'name': venue['name'],
                'latitude': venue['geocodes']['main']['latitude'],
                'longitude': venue['geocodes']['main']['longitude'],
                'rating': venue.get('rating', 'No rating')  # Use 'No rating' if rating is not available
            }
            parsed_venues.append(venue_details)
        return parsed_venues
    else:
        print(f"Failed to fetch POIs: {response.status_code}")
        return []

# Fetch and parse POIs for a sample station
example_pois = fetch_and_parse_pois(stations_df.iloc[0]['latitude'], stations_df.iloc[0]['longitude'])

# Notes:
# - API key is required for authentication with the Foursquare API.
# - The function requests POIs around a given location within a 1000m radius.
# - Parses each POI to extract and return a simplified dictionary of details.
# - Handles cases where a POI does not have a rating.
# - Checks the status code of the response to handle failed requests.

Put your parsed results into a DataFrame

In [ ]:
all_pois = []

for _, row in stations_df.iterrows():
    pois = fetch_and_parse_pois(row['latitude'], row['longitude'])
    all_pois.extend(pois)  # Combine POIs from all stations

pois_df = pd.DataFrame(all_pois)

# Notes:
# - Iterates through each bike station to fetch nearby POIs.
# - Combines all fetched POIs into a single list of dictionaries.
# - Converts this list into a DataFrame for easy handling and analysis.

# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [ ]:
#Unable to accsess YelpAPI

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [ ]:
#Unable to accsess YelpAPI

Put your parsed results into a DataFrame

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

I ran into issues with Foursquare data; it lacked detailed info like ratings, and I also couldn't access the Yelp API because I wasn't getting the verification emails needed to create an API.

Get the top 10 restaurants according to their rating

In [10]:
#Unable to accsess YelpAPI (Foursquare results only)
API_KEY = 'fsq3hhg8hTQCfL/aPuz1PSLab+R48Pg6oojSQ2NqplkdW8A='

# Coordinates for Barcelona. This latitude and longitude are used as the center point for the search query.
LAT, LON = '41.3851', '2.1734'

# Headers for the API request. 'Accept' specifies the format of the response we expect (JSON in this case).
# 'Authorization' is used to pass the API key for authentication.
headers = {
    "Accept": "application/json",
    "Authorization": API_KEY
}

# Parameters for the API request. 'll' stands for latitude and longitude, specifying the search location.
# 'radius' defines the search area's radius in meters. 'limit' is the maximum number of results to return.
# 'categories' is used to filter search results to a specific category, in this case, restaurants.
params = {
    "ll": f"{LAT},{LON}",
    "radius": 1000,
    "limit": 50,
    "categories": "13065"  # This category ID is supposed to represent restaurants.
}

# Sending a GET request to Foursquare's search API endpoint with the specified headers and parameters.
response = requests.get("https://api.foursquare.com/v3/places/search", headers=headers, params=params)

# Checking the status code of the response to ensure the request was successful.
if response.status_code == 200:
    # Extracting the 'results' from the JSON response.
    places = response.json().get("results", [])
    # Diagnostic print to show how many places were returned by the query.
    print(f"Places returned: {len(places)}")
    
    # Filtering the list of places to include only those that have a 'rating'.
    rated_places = [place for place in places if 'rating' in place]
    # Diagnostic print to show how many of the returned places have ratings.
    print(f"Rated places returned: {len(rated_places)}")
    
    # If there are places with ratings, sort them by the 'rating' key in descending order and take the top 10.
    if rated_places:
        top_rated = sorted(rated_places, key=lambda x: x['rating'], reverse=True)[:10]
        # Printing the top 10 rated places, showing their name and rating.
        for index, place in enumerate(top_rated, 1):
            print(f"{index}. {place['name']} - Rating: {place.get('rating', 'No rating')}")
    else:
        # Message indicating that no places with ratings were found.
        print("No places with ratings were found.")
else:
    # If the request was not successful, print the error status code.
    print("Error:", response.status_code)

Places returned: 50
Rated places returned: 0
No places with ratings were found.
